In [90]:
import pandas as pd

In [100]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})

In [101]:
products

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16
4,2,65,2019-08-17
5,3,20,2019-08-18


Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |

In [93]:
products_filtered=products.loc[products["change_date"] <= pd.to_datetime("2019-08-16")].copy()
products_filtered

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16


In [94]:
products_filtered["rank"]=products_filtered.groupby(by="product_id")["change_date"] \
    .rank(method="max", ascending=False)
products_filtered

,product_id,new_price,change_date,rank
0,1,20,2019-08-14,3.0
1,2,50,2019-08-14,1.0
2,1,30,2019-08-15,2.0
3,1,35,2019-08-16,1.0


In [95]:
products_filtered.loc[products_filtered["rank"]==1].drop(columns=["change_date","rank"],axis=1)

,product_id,new_price
1,2,50
3,1,35


In [96]:
product_distinct=pd.DataFrame(products["product_id"].drop_duplicates())
product_distinct

,product_id
0,1
1,2
5,3


In [97]:
product_distinct.merge(right=products_filtered.loc[products_filtered["rank"]==1].drop(columns=["change_date","rank"],axis=1), on="product_id", how="left") \
    .fillna({"new_price":10}) \
    .rename(columns={"new_price":"price"})

,product_id,price
0,1,35
1,2,50
2,3,10


# Better solution

In [105]:
df1 = products[products.change_date <= '2019-08-16']
df2 = products[products.change_date > '2019-08-16'].copy()
df2.new_price = 10
df2 = df2[~df2.product_id.isin(df1.product_id)]
pd.concat([df1,df2]).sort_values(by = 'change_date', ascending = False).drop_duplicates(['product_id'])[['product_id','new_price']].rename(columns = {'new_price' : 'price'})

,product_id,price
5,3,10
3,1,35
1,2,50


Additional debugging

In [98]:
data=[
    [3, 64, "2019-08-08"],
    [3, 72, "2019-07-22"],
    [3, 46, "2019-10-14"],
    [3, 72, "2019-08-31"],
    [3, 21, "2019-06-19"],
    [3, 23, "2019-08-07"]
]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})
products

,product_id,new_price,change_date
0,3,64,2019-08-08
1,3,72,2019-07-22
2,3,46,2019-10-14
3,3,72,2019-08-31
4,3,21,2019-06-19
5,3,23,2019-08-07


In [99]:
products_filtered=products.loc[products["change_date"] <= pd.to_datetime("2019-08-16")].copy()

products_filtered["rank"]=products_filtered.groupby(by="product_id")["change_date"] \
    .rank(method="max", ascending=False)
products_filtered

,product_id,new_price,change_date,rank
0,3,64,2019-08-08,1.0
1,3,72,2019-07-22,3.0
4,3,21,2019-06-19,4.0
5,3,23,2019-08-07,2.0
